In [3]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import json
from pathlib import Path
import torch
import time

model_name = 'cross-encoder/ms-marco-MiniLM-L-12-v2'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

split = 'val'
nlq_list = json.loads(Path(f"/data/soyeonhong/GroundVQA/data/unified/annotations.NLQ_{split}.json").read_text())

In [4]:
except_list = ['70a350cd-4f32-40ed-80dd-23a48e7c4e46',
               '04199001-307a-40fd-b20c-4b4128546b89'] # nlq_train

In [6]:
for annotation in nlq_list:
    
    video_id = annotation['video_id']
    sample_id = annotation['sample_id']
    question = annotation['question']
    
    print(sample_id)
    
    if not video_id in except_list:
        llava_caption = json.loads(Path(f"/data/soyeonhong/GroundVQA/llava-v1.6-34b/global/{video_id}.json").read_text())['answers']
        
        caption_list = []
        query_list = []
        frame_idx_list = []
        for caption in llava_caption:
            frame_idx_list.append(caption[0])
            caption_list.append(caption[2])
            query_list.append(nlq_list[0]['question'])
            
        features = tokenizer(query_list, caption_list, max_length= 172, return_tensors='pt', padding=True, truncation=True)
        
        model.eval()
        with torch.no_grad():
            output = model(**features, return_dict=True, output_hidden_states=True)
        
        pool_result = model.bert.pooler(output.hidden_states[-1])
        
        save_list = []
        for frame_idx, result in zip(frame_idx_list, pool_result):
            save_list.append((frame_idx, result))
            
        torch.save(save_list, f"/data/soyeonhong/GroundVQA/cross_encoding/{split}/{sample_id}.pt")
        end = time.time()

95441b96-e2ef-4e61-b74f-10ebc2da5b4e_0
Time: 1.7426888942718506
95441b96-e2ef-4e61-b74f-10ebc2da5b4e_1
Time: 1.4834678173065186
95441b96-e2ef-4e61-b74f-10ebc2da5b4e_2
Time: 1.4606733322143555
95441b96-e2ef-4e61-b74f-10ebc2da5b4e_3
Time: 1.4502089023590088
95441b96-e2ef-4e61-b74f-10ebc2da5b4e_4
Time: 1.4614267349243164
95441b96-e2ef-4e61-b74f-10ebc2da5b4e_5
Time: 1.441025733947754
95441b96-e2ef-4e61-b74f-10ebc2da5b4e_6
Time: 1.4726204872131348
95441b96-e2ef-4e61-b74f-10ebc2da5b4e_7
Time: 1.4606902599334717
95441b96-e2ef-4e61-b74f-10ebc2da5b4e_8
Time: 1.4280025959014893
95441b96-e2ef-4e61-b74f-10ebc2da5b4e_9
Time: 1.4874603748321533
9f76d2cb-17f9-4faf-9e4c-d05c9b38b42c_0
Time: 1.5095272064208984
9f76d2cb-17f9-4faf-9e4c-d05c9b38b42c_1
Time: 1.4144268035888672
9f76d2cb-17f9-4faf-9e4c-d05c9b38b42c_2
Time: 1.5561957359313965
9f76d2cb-17f9-4faf-9e4c-d05c9b38b42c_3
Time: 1.4328045845031738
9f76d2cb-17f9-4faf-9e4c-d05c9b38b42c_4
Time: 1.6417837142944336
9f76d2cb-17f9-4faf-9e4c-d05c9b38b42c_5
Ti

KeyboardInterrupt: 